# Importação de Bibliotecas

In [1]:
pip install ucimlrepo

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd

# Carregar e preparar o Dataset

In [3]:
# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

df = pd.concat([X, y], axis=1)

In [6]:
print(df["Cover_Type"].value_counts())

Cover_Type
2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: count, dtype: int64


In [7]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


# Normalização das variáveis quantitativas

In [8]:
# Listando as colunas quantitativas do dataset
colunas_quantitativas = [
    "Elevation", "Aspect", "Slope",
    "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways", "Hillshade_9am",
    "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"
]

print("Antes da normalização:")
print(df[colunas_quantitativas].describe())

Antes da normalização:
           Elevation         Aspect          Slope  \
count  581012.000000  581012.000000  581012.000000   
mean     2959.365301     155.656807      14.103704   
std       279.984734     111.913721       7.488242   
min      1859.000000       0.000000       0.000000   
25%      2809.000000      58.000000       9.000000   
50%      2996.000000     127.000000      13.000000   
75%      3163.000000     260.000000      18.000000   
max      3858.000000     360.000000      66.000000   

       Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  \
count                     581012.000000                   581012.000000   
mean                         269.428217                       46.418855   
std                          212.549356                       58.295232   
min                            0.000000                     -173.000000   
25%                          108.000000                        7.000000   
50%                          218.000000 

In [9]:
# Criando o normalizador (StandardScaler)
scaler = StandardScaler()

# Aplicando a normalização nas colunas quantitativas
df[colunas_quantitativas] = scaler.fit_transform(df[colunas_quantitativas])

print("\nDepois da normalização:")
print(df[colunas_quantitativas].describe())


Depois da normalização:
          Elevation        Aspect         Slope  \
count  5.810120e+05  5.810120e+05  5.810120e+05   
mean  -3.819486e-16 -3.913408e-17  6.417988e-17   
std    1.000001e+00  1.000001e+00  1.000001e+00   
min   -3.930094e+00 -1.390866e+00 -1.883448e+00   
25%   -5.370487e-01 -8.726087e-01 -6.815629e-01   
50%    1.308455e-01 -2.560618e-01 -1.473917e-01   
75%    7.273071e-01  9.323547e-01  5.203224e-01   
max    3.209587e+00  1.825901e+00  6.930377e+00   

       Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  \
count                      5.810120e+05                    5.810120e+05   
mean                      -7.122402e-17                    8.061620e-17   
std                        1.000001e+00                    1.000001e+00   
min                       -1.267604e+00                   -3.763928e+00   
25%                       -7.594865e-01                   -6.761941e-01   
50%                       -2.419591e-01                   -2.8165

# Dividir em Treino e Teste

In [10]:
# Ajustar a variável alvo para começar de 0
df["Cover_Type"] = df["Cover_Type"] - 1

print("Valores únicos do target após ajuste:", df["Cover_Type"].unique())

Valores únicos do target após ajuste: [4 1 0 6 2 5 3]


In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"]  # Target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

Tamanho do conjunto de treino: (464809, 54)
Tamanho do conjunto de teste: (116203, 54)


# Converter para Tensors

In [12]:
import torch

# Convertendo o df para Tensors do PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
print(f"y_test_tensor shape: {y_test_tensor.shape}")


X_train_tensor shape: torch.Size([464809, 54])
X_test_tensor shape: torch.Size([116203, 54])
y_train_tensor shape: torch.Size([464809])
y_test_tensor shape: torch.Size([116203])


# Criar DataLoader

In [13]:
from torch.utils.data import TensorDataset, DataLoader

# Criando o dataset do PyTorch
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Criando os DataLoaders
batch_size = 64  # Número de amostras por lote
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Número de batches no conjunto de treino: {len(train_loader)}")
print(f"Número de batches no conjunto de teste: {len(test_loader)}")


Número de batches no conjunto de treino: 7263
Número de batches no conjunto de teste: 1816


# Definir a Rede Neural

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# arquitetura da rede neural
class CoverTypeNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(CoverTypeNN, self).__init__()

        # Camada de entrada -> Primeira camada oculta
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)  # Normalização

        # Primeira -> Segunda camada oculta
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2)  # Normalização

        # Segunda camada oculta -> Saída
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x))) 
        x = F.relu(self.bn2(self.fc2(x))) 
        x = self.fc3(x)  # Saída (sem ativação pois usaremos CrossEntropyLoss)
        return x

# Definição dos tamanhos das camadas
input_size = X_train_tensor.shape[1]  # Número de colunas (54)
hidden_size1 = 128
hidden_size2 = 64
output_size = 7  # 7 classes de cobertura florestal

# Criando o modelo
model = CoverTypeNN(input_size, hidden_size1, hidden_size2, output_size)
print(model)


CoverTypeNN(
  (fc1): Linear(in_features=54, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=7, bias=True)
)


# Configurar Função de Perda e Otimizador

In [15]:
import torch.optim as optim

# Definição da função de perda (CrossEntropyLoss para classificação)
criterion = nn.CrossEntropyLoss()

# Definição do otimizador (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Taxa de aprendizado = 0.001

# Move o modelo para a GPU se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Modelo configurado para treinar em: {device}")


Modelo configurado para treinar em: cpu


# Treinamento do Modelo

In [16]:
# Definir número de Epochs
num_epochs = 10

# treinamento
for epoch in range(num_epochs):
    model.train()

    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        # Mover os dados para a CPU
        batch_X, batch_y = batch_X.to("cpu"), batch_y.to("cpu")

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)

        # Calcular perda
        loss = criterion(outputs, batch_y)

        # Backward pass (cálculo dos gradientes) e atualização dos pesos
        loss.backward()
        optimizer.step()

        # Acumular perda
        running_loss += loss.item()

    # Exibir estatísticas da época
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")



Epoch [1/10], Loss: 0.5166
Epoch [2/10], Loss: 0.4248
Epoch [3/10], Loss: 0.3958
Epoch [4/10], Loss: 0.3791
Epoch [5/10], Loss: 0.3674
Epoch [6/10], Loss: 0.3581
Epoch [7/10], Loss: 0.3526
Epoch [8/10], Loss: 0.3465
Epoch [9/10], Loss: 0.3416
Epoch [10/10], Loss: 0.3376


# Avaliação do Modelo

In [17]:
# Colocar o modelo em modo de avaliação
model.eval()

# Inicializar contadores
correct = 0
total = 0

# Desativar cálculo de gradiente (econ)
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        # Mover para CPU/GPU conforme necessário
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Fazer previsões
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)  # Obtém a classe com maior probabilidade

        # Atualizar contadores
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

# Calcular acurácia
accuracy = 100 * correct / total
print(f"Acurácia no conjunto de teste: {accuracy:.2f}%")


Acurácia no conjunto de teste: 88.06%


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Fazer previsões no conjunto de teste
y_pred_tensor = model(X_test_tensor)
y_pred = torch.argmax(y_pred_tensor, axis=1).numpy()

# Relatório de classificação (precision, recall, F1-score)
print("Relatório de Classificação:\n")
print(classification_report(y_test, y_pred))


Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.90      0.84      0.87     42368
           1       0.87      0.93      0.90     56661
           2       0.87      0.88      0.88      7151
           3       0.82      0.77      0.79       549
           4       0.81      0.58      0.67      1899
           5       0.79      0.75      0.77      3473
           6       0.91      0.89      0.90      4102

    accuracy                           0.88    116203
   macro avg       0.85      0.81      0.83    116203
weighted avg       0.88      0.88      0.88    116203



O modelo está bom no geral (88% de acurácia), mas está errando mais nas classes menores (3, 4 e 5). Isso acontece porque elas têm menos exemplos, então o modelo não aprende tão bem.

# Aplicar SMOTE

In [19]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Aplicar SMOTE para balancear os dados
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Verificando a nova distribuição das classes
print("Distribuição após SMOTE:", Counter(y_resampled))


Distribuição após SMOTE: Counter({1: 226640, 6: 226640, 0: 226640, 4: 226640, 2: 226640, 3: 226640, 5: 226640})


# Conversão para Tensors e Split de Dados

In [ ]:
# Convertendo os dados balanceados para tensors
X_resampled_tensor = torch.tensor(X_resampled.values, dtype=torch.float32)
y_resampled_tensor = torch.tensor(y_resampled.values, dtype=torch.long)

# Dividindo os dados balanceados em treino e teste
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_resampled_tensor, y_resampled_tensor, test_size=0.2, random_state=42, stratify=y_resampled_tensor
)

# Atualização dos DataLoaders

In [21]:
# Criando os datasets do PyTorch com os dados balanceados pelo SMOTE
train_dataset = TensorDataset(X_train_final, y_train_final)
test_dataset = TensorDataset(X_test_final, y_test_final)


# Já existiam DataLoaders antes do SMOTE
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [22]:
# Treinamento do modelo
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/10], Loss: 0.2902
Epoch [2/10], Loss: 0.2618
Epoch [3/10], Loss: 0.2495
Epoch [4/10], Loss: 0.2425
Epoch [5/10], Loss: 0.2362
Epoch [6/10], Loss: 0.2323
Epoch [7/10], Loss: 0.2285
Epoch [8/10], Loss: 0.2255
Epoch [9/10], Loss: 0.2229
Epoch [10/10], Loss: 0.2202


In [23]:
from sklearn.metrics import classification_report, accuracy_score

# Avaliação do modelo
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(batch_y.numpy())
        y_pred.extend(predicted.numpy())

# Cálculo da acurácia
test_accuracy = accuracy_score(y_true, y_pred)
print(f'Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%')

Acurácia no conjunto de teste: 93.54%


In [24]:
# Relatório de classificação
print("Relatório de Classificação:\n")
print(classification_report(y_true, y_pred))

Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.91      0.83      0.87     45328
           1       0.86      0.86      0.86     45328
           2       0.94      0.93      0.94     45328
           3       0.98      0.99      0.99     45328
           4       0.95      0.99      0.97     45328
           5       0.93      0.95      0.94     45328
           6       0.97      1.00      0.98     45328

    accuracy                           0.94    317296
   macro avg       0.93      0.94      0.93    317296
weighted avg       0.93      0.94      0.93    317296



O modelo apresentou uma melhora significativa na acurácia, atingindo 93,54%, e agora trabalha com classes balanceadas. O recall das classes menores aumentou, tornando a classificação mais equilibrada e reduzindo vieses anteriores. Além disso, as classes com melhor desempenho, como as de maior recall (3, 4 e 6), indicam que o modelo está aprendendo bem padrões distintos.